In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from orbit.models import LGT, DLT
from orbit.utils.dataset import load_iclaims

from orbit.utils.params_tuning import grid_search_orbit
from orbit.diagnostics.metrics import smape, mape, wmape

## Load Data

In [2]:
raw_data = load_iclaims()
data = raw_data.copy()

print(data.shape)
data.head(5)

(443, 7)


,week,claims,trend.unemploy,trend.filling,trend.job,sp500,vix
0,2010-01-03,13.386595,0.219882,-0.318452,0.117500,-0.417633,0.122654
1,2010-01-10,13.624218,0.219882,-0.194838,0.168794,-0.425480,0.110445
2,2010-01-17,13.398741,0.236143,-0.292477,0.117500,-0.465229,0.532339
3,2010-01-24,13.137549,0.203353,-0.194838,0.106918,-0.481751,0.428645
4,2010-01-31,13.196760,0.134360,-0.242466,0.074483,-0.488929,0.487404


## LGT Tuning Exampe

<div class="alert alert-info">

To have a consistent outlook here, users need to make sure an updated version of `ipywidgets` is installed.

</div>

In [3]:
lgt = LGT(
    date_col='week',
    response_col='claims',
    seasonality=52,
    estimator='stan-map',
    verbose=False
)

In [4]:
param_grid = {'level_sm_input': [0.3, 0.5, 0.8],
              'seasonality_sm_input': [0.3, 0.5, 0.8]}

### Backtest

In [5]:
best_params, tuned_df = grid_search_orbit(
    param_grid,                           
    model=lgt,                           
    df=data,                           
    min_train_len=100,                         
    incremental_len=100,                    
    forecast_len=20,                        
    eval_method='backtest',                           
    verbose=False
)

  0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
tuned_df.head()

,level_sm_input,seasonality_sm_input,metrics
0,0.3,0.3,0.006624
1,0.3,0.5,0.006012
2,0.3,0.8,0.005633
3,0.5,0.3,0.007426
4,0.5,0.5,0.006844


In [7]:
best_params

[{'level_sm_input': 0.3, 'seasonality_sm_input': 0.8}]

### BIC

Users can also tune the model using BIC by specifying `eval_method='bic'`.

In [8]:
best_params, tuned_df = grid_search_orbit(
    param_grid,                      
    model=lgt,                       
    df=data,     
    min_train_len=100,             
    incremental_len=100,            
    forecast_len=20,               
    eval_method='bic',                             
    verbose=False
)

  0%|          | 0/9 [00:00<?, ?it/s]

In [9]:
best_params

[{'level_sm_input': 0.3, 'seasonality_sm_input': 0.8}]

## DLT Tuning Example

In [10]:
dlt = DLT(
    date_col='week',
    response_col='claims',
    regressor_col=['trend.unemploy', 'trend.filling', 'trend.job'],
    seasonality=52,
    estimator='stan-map',
    verbose=False
)

In [11]:
param_grid = {'damped_factor': [0.3, 0.5, 0.8],
              'slope_sm_input': [0.3, 0.5, 0.8]}

In [12]:
best_params, tuned_df = grid_search_orbit(
    param_grid,
    model=dlt,
    df=data,
    min_train_len=100,                
    incremental_len=100,                  
    forecast_len=20,                     
    eval_method='backtest',                 
    verbose=False
)

  0%|          | 0/9 [00:00<?, ?it/s]

In [13]:
tuned_df.head()

,damped_factor,slope_sm_input,metrics
0,0.3,0.3,0.005776
1,0.3,0.5,0.005826
2,0.3,0.8,0.005813
3,0.5,0.3,0.005788
4,0.5,0.5,0.005637


In [14]:
best_params

[{'damped_factor': 0.5, 'slope_sm_input': 0.5}]